## K Nearest Neighbors

In [17]:
import os
import settings
import pandas as pd
import numpy as np
from scipy.spatial import distance


In [18]:
def read_data():
    df = pd.read_excel(os.path.join(settings.PROCESSED_DIR, "all_with_liwc_segmented.xls"), encoding="ISO-8859-1")
    return df

In [19]:
df = read_data()

In [20]:
def find_similar_speaker(name):
    numerical = df.select_dtypes(include=['int64','float64'])
    df_normalized = (numerical - numerical.mean()) / numerical.std()
    del df_normalized['music']
    del df_normalized['conversation']
    speaker_normalized = df_normalized[df["main_speaker"] == name]
    speaker_normalized = speaker_normalized.iloc[0]
    euclidean_distances = df_normalized.apply(lambda row: distance.euclidean(row, speaker_normalized), axis=1)
    print(euclidean_distances.nsmallest(5))
    second_smallest_value = euclidean_distances.nsmallest(2).iloc[1]
    rec_idx = euclidean_distances[euclidean_distances == second_smallest_value].index
    return df[['main_speaker', 'description', 'url' ]].iloc[rec_idx]

In [21]:
find_similar_speaker('E.O. Wilson')

64     0.000000
81     7.003069
389    7.866872
54     8.246570
224    8.375643
dtype: float64


,main_speaker,description,url
85,Sheila Patek,Biologist Sheila Patek talks about her work me...,https://www.ted.com/talks/sheila_patek_clocks_...


In [23]:
def find_similar_talk_moral(title):
    numerical = df[['HarmVirtue', 'HarmVice', 'AuthorityVirtue', 'AuthorityVice', 'IngroupVirtue', 'IngroupVice', 'FairnessVirtue', 'FairnessVice', 'PurityVirtue', 'PurityVice', 'MoralityGeneral']]
    df_normalized = (numerical - numerical.mean()) / numerical.std()
    speaker_normalized = df_normalized[df["title"] == title]
    speaker_normalized = speaker_normalized.iloc[0]
    print(speaker_normalized)
    euclidean_distances = df_normalized.apply(lambda row: distance.euclidean(row, speaker_normalized), axis=1)
    print(euclidean_distances.nsmallest(5))
    second_smallest_value = euclidean_distances.nsmallest(2).iloc[1]
    rec_idx = euclidean_distances[euclidean_distances == second_smallest_value].index
    #print(euclidean_distances.sort_values())
    return df[['main_speaker', 'description', 'url']].iloc[rec_idx]

In [24]:
find_similar_talk_moral('Our loss of wisdom')

HarmVirtue         0.658628
HarmVice          -0.475024
AuthorityVirtue    0.892933
AuthorityVice      0.374477
IngroupVirtue      0.055593
IngroupVice       -0.436870
FairnessVirtue     0.303447
FairnessVice      -0.243597
PurityVirtue      -0.070717
PurityVice        -0.161798
MoralityGeneral    3.856452
Name: 384, dtype: float64
384     0.000000
1976    1.498020
2451    1.607556
1049    1.693742
1596    1.743696
dtype: float64


,main_speaker,description,url
2163,Arthur Brooks,Conservatives and liberals both believe that t...,https://www.ted.com/talks/arthur_brooks_a_cons...
